# Устанавливаем всё необходимое

In [186]:
#!g1.1
!git clone https://github.com/ivan7022/MakeUHappyChatBot
%cd MakeUHappyChatBot

fatal: destination path 'MakeUHappyChatBot' already exists and is not an empty directory.


Exception: Process exited with code 128

In [187]:
#!g1.1
%pwd

'/home/jupyter/work/resources'

In [160]:
#!g1.1
%pip install transformers
%pip install emoji
%pip install google_trans_new

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.

Usage:   
  /usr/bin/python3.7 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/bin/python3.7 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/bin/python3.7 -m pip install [options] [-e] <vcs project url> ...
  /usr/bin/python3.7 -m pip install [options] [-e] <local project path> ...
  /usr/bin/python3.7 -m pip install [options] <archive url/path> ...

no such option: --update
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
#!g1.1


In [176]:
#!g1.1
!ls

README.md  config.json	offset.txt


# Код для поддержки бота

In [174]:
#!g1.1
import torch
import pickle
from transformers import AutoModelForCausalLM, AutoTokenizer
class DialoGPT:
    tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
    model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
    chat_history = {}

    def get_length_param(self, text: str) -> str:
        tokens_count = len(self.tokenizer.encode(text))
        if tokens_count <= 15:
            len_param = '1'
        elif tokens_count <= 50:
            len_param = '2'
        elif tokens_count <= 256:
            len_param = '3'
        else:
            len_param = '-'
        return len_param

    """
    Возвращает ответ на текст, введённый пользователем
    @param input_user Текст, введённый пользователе
    @param chat_id id чата, куда был введен ответ или -1, если чата нет
    @returns Ответ на текст, введённый пользователем
    """
    def get_response(self, input_user, chat_id):
        input_user = input_user[:256]
        tokenizer = self.tokenizer
        
        # encode the new user input, add parameters and return a tensor in Pytorch
        new_user_input = f"|0|{self.get_length_param(input_user)}|" + input_user + tokenizer.eos_token +  "|1|1|"
        new_user_input_ids = tokenizer.encode(new_user_input, return_tensors="pt")

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([self.chat_history[chat_id], new_user_input_ids], dim=-1) if chat_id in self.chat_history else new_user_input_ids

        # generated a response
        self.chat_history[chat_id] = self.model.generate(
            bot_input_ids,
            num_return_sequences=1,
            max_length=512,
            no_repeat_ngram_size=3,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature = 0.6,
            mask_token_id=tokenizer.mask_token_id,
            eos_token_id=tokenizer.eos_token_id,
            unk_token_id=tokenizer.unk_token_id,
            pad_token_id=tokenizer.pad_token_id,
            device='cuda',
        )
        #print(len(self.chat_history[chat_id]))
        
        result = tokenizer.decode(self.chat_history[chat_id][:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        if chat_id == -1:
            self.restart(-1)
        
        return result
    
    def restart(self, chat_id):
        if chat_id in self.chat_history:
            del self.chat_history[chat_id]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [179]:
#!g1.1
%cd ..

In [181]:
#!g1.1
%cd ..

/home/jupyter/work/resources


Cloning into 'MakeUHappyChatBot'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 64 (delta 15), reused 43 (delta 8), pack-reused 0
Unpacking objects: 100% (64/64), done.


## Проверяем, что всё ок

In [175]:
#!g1.1
chat_bot = DialoGPT()
chat_bot.get_response('привет', -1)

1


'Я рад'

## Дополнительные функции

In [ ]:
#!g1.1


In [172]:
#!g1.1
import emoji
import unicodedata
from google_trans_new import google_translator

translator = google_translator()  
def eng_to_rus(text):
    return translator.translate(text,lang_tgt='ru')

def emoji_to_text(emoji):
    return eng_to_rus(unicodedata.name(emoji).lower())

def de_emojify(text):
    result = ''
    for sym in text:
        if emoji.emoji_count(sym):
            result += " " + emoji_to_text(sym) + " "
        else:
            result += sym
    return ' '.join(filter(bool, result.split(' ')))

print(de_emojify('Привет 👍 \nКак дела? 😀😀'))

Привет палец вверх знака 
Как дела? ухмыляясь лицом ухмыляясь лицом


In [173]:
#!g1.1
def extract_text(message):
    if 'text' in message:
        return message['text']
    elif 'caption' in message:
        return message['caption']
    elif 'sticker' in message:
        return message['sticker']['emoji']
    else:
        return ''

"""
Получает текст из сообщения
@param message Получаемое сообщение из Telegram API
@returns Текст из сообщения
"""
def get_text(message):
    return de_emojify(extract_text(message))
    
"""
Обрабатывает команду
@param text Текст, в котором может содержаться команда
@param chat_id id чата

@returns Пару (итоговый текст, отправлять ли этот текст на обработку ботом)
"""
def process_command(text, chat_id):
    start = '/start'
    restart = '/restart'
    ref = '/make_u_happy_bot'
    
    if text.startswith(start):
        return 'Привет! Хочешь о чём-нибудь поговорить?', False
    elif text.startswith(restart):
        chat_bot.restart(chat_id)
        return 'История диалога очищена', False
    elif text.startswith(ref):
        return text[len(ref):], True
    else:
        return text, True

# Основной цикл работы бота

In [171]:
#!g1.1
import requests

offset_file = open('offset.txt', 'r+')
offset = int(offset_file.read())
print('Started')
while True:
    get_url = 'https://api.telegram.org/bot1853926901:AAHs-iYO-hp559x52L67egfEZ8FzLo26WAk/getUpdates?offset=' + str(offset)
    result = requests.get(get_url).json()

    if len(result['result']) == 0:
        continue
    #print(result)

    result = result['result'][0]
    if 'message' in result:
        message = result['message']
        chat_id = message['chat']['id']

        text = get_text(message)
        print(text)
        text, should_send = process_command(text, chat_id)
        
        if should_send:
            text = chat_bot.get_response(text, chat_id)

        if len(text) > 0:
            send_url = 'https://api.telegram.org/bot1853926901:AAHs-iYO-hp559x52L67egfEZ8FzLo26WAk/sendMessage?chat_id=' \
                + str(chat_id) + '&text=' + text 
            requests.get(send_url)
    offset = int(result['update_id']) + 1
    offset_file.truncate()
    offset_file.write(str(offset))

Started
ghbdtn
привет
давай говорить на русском
и что?
привет
Серьезное лицо с символами, покрывающими рот


KeyboardInterrupt: 

In [105]:
#!g1.1
import pickle
d = {
    'a':5,
    'b':1
}

f = open('test', 'wb')
pickle.dump(d, f)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:815: UserWarning: The following variables cannot be serialized: f
  warnings.warn(message)


In [ ]:
#!g1.1
